In [2]:
import pandas as pd
import seaborn as sns
import plotly.express as px
import matplotlib.pyplot as plt
import os

print(os.listdir("../data"))
PREC_PATH = "../data/preprocessed/{}.csv"
PATH = "../data/{}.csv"

games = pd.read_csv(PREC_PATH.format("games"))
details = pd.read_csv(PREC_PATH.format("details"))
players = pd.read_csv(PATH.format("players"))
ranking = pd.read_csv(PREC_PATH.format("rankings"))
teams = pd.read_csv(PATH.format("teams"))

['.gitattributes', 'games.csv', 'games_details.csv', 'players.csv', 'preprocessed', 'ranking.csv', 'teams.csv']


C:\Users\mathi\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [13]:
player_selection = details.groupby("player_name").mean()[["player_id", "pts"]].reset_index()\
    .merge(players, right_on="PLAYER_ID", left_on="player_id")\
    .sort_values("pts", ascending=False)\
    .drop_duplicates("player_name")\
    .head(100)\
    .merge(teams[["TEAM_ID", "ABBREVIATION"]], on="TEAM_ID")\
    [["player_name", "player_id", "pts"]]


C:\Users\antom\miniconda3\lib\site-packages\pandas\core\reshape\merge.py:1204: UserWarning: You are merging on int and float columns where the float values are not equal to their int representation
  warnings.warn(


In [16]:
player_selection.to_csv("../website/data/player_selection.csv")

In [15]:
os.listdir("../..")

['.idea',
 'datavis-project-2022-lebron-jenkins',
 'dataviz-exercise-02',
 'dataviz-exercise-03',
 'DataVizExercise01',
 'idea.txt',
 'intro.html']

# Teams

Take the teamId the name and the abreviation

In [9]:
teams_summary = teams[['TEAM_ID','ABBREVIATION', 'NICKNAME']]
teams_summary.to_csv("../website/data/teams_summary.csv")

# Games

In [15]:
games[['SEASON']].drop_duplicates("SEASON").sort_values("SEASON", ascending=False)

,SEASON
0,2021-01-01
1076,2020-01-01
2325,2019-01-01
18698,2018-01-01
20076,2017-01-01
21458,2016-01-01
22863,2015-01-01
24279,2014-01-01
3203,2013-01-01
4630,2012-01-01
